In [0]:
%run ../config

In [0]:
spark.sql(f'USE CATALOG {catalog}')
spark.sql(f'USE SCHEMA {db}')


In [0]:
from pyspark.sql.functions import row_number, concat, lpad, col, lit
from pyspark.sql.window import Window

shipment_recommendations = spark.table('shipment_recommendations')

filtered = shipment_recommendations.filter(col("qty_shipped") > 0)
unique_pairs = (
    filtered.select("stock_location_id", "designator_id")
    .distinct()
    .withColumn(
        "rn",
        row_number().over(Window.orderBy("stock_location_id", "designator_id"))
    )
)

parts_orders = (
    filtered.join(
        unique_pairs,
        on=["stock_location_id", "designator_id"],
        how="inner"
    )
    .withColumn(
        "order_number",
        concat(
            lit("PR-"),
            lpad(col("rn").cast("string"), 6, "0")
        )
    )
    .drop('rn')
)

In [0]:
parts_orders.write.mode("overwrite").saveAsTable("ai_part_orders")

In [0]:
status = spark.table('ship_current_status_gold')
parts = spark.table('parts_silver').select('NSN', 'type', 'sensors').distinct()

In [0]:

query = """
WITH parts_ex AS (
  SELECT type, EXPLODE(sensors) AS sensor
  FROM parts_meta
)
SELECT
  scsg.turbine_id,
  scsg.prediction,
  COLLECT_LIST(parts_ex.type) AS parts_required
FROM ship_current_status_gold scsg
JOIN parts_ex
  ON scsg.prediction = parts_ex.sensor
WHERE scsg.prediction != 'ok'
GROUP BY scsg.turbine_id, scsg.prediction
"""
df = spark.sql(query)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id, row_number, format_string, when, collect_set
from pyspark.sql.window import Window

window = Window.orderBy(monotonically_increasing_id())
df = df.withColumn(
    "work_order",
    format_string("AI-WO %03d", row_number().over(window))
)

scsg = spark.table('ship_current_status_gold')

ai_wo = scsg.join(
    df.select("turbine_id", "prediction", "parts_required", "work_order"),
    on=["turbine_id", "prediction"],
    how="inner"
)

ai_wo = ai_wo.withColumn(
    "priority",
    when(ai_wo.maintenance_type == "Organizational Level", "Routine")
    .when(ai_wo.maintenance_type == "Intermediate Level", "Urgent")
    .when(ai_wo.maintenance_type == "Depot Level", "CASREP")
    .otherwise(None)
)


# Example: group by a column and collapse another column's values into a list
collapsed_df = parts_orders.groupBy("designator").agg(collect_set("order_number").alias("part_orders"))

ai_wo = ai_wo.join(collapsed_df, on="designator", how="inner")

display(ai_wo)

ai_wo.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("ai_work_orders")


## Parts orders
